In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

In [5]:
trade = pd.read_csv("../data/trade-information.csv", index_col=False)
ir = pd.read_csv("../data/trade-price-ir-vegas.csv", index_col=False)
swap = pd.read_csv("../data/market-data-swap-rates.csv", index_col=False)
swaption = pd.read_csv("../data/market-data-swaption-vols.csv", index_col=False)

In [6]:
trade['bound width'] = trade['upper_bound'] - trade['lower_bound'] 

trade['month_pay_frequency'] = trade['pay_frequency'].str.removesuffix("M") # remove M from string
trade['month_pay_frequency'] = trade['month_pay_frequency'].astype(int) # convert to int


trade['year_maturity'] = trade['maturity'].str.removesuffix("Y") # remove Y from string
trade['year_maturity'] = trade['year_maturity'].astype(int)  # convert to int


In [7]:
trade

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397,0.0353,4,10


In [8]:
swap

,Date,Start Date,Tenor,Swap Rate
0,2021-01-13,2022-01-13,10y,1.081333
1,2021-01-13,2022-01-13,15y,1.265960
2,2021-01-13,2022-01-13,18M,0.221625
3,2021-01-13,2022-01-13,1y,0.195474
4,2021-01-13,2022-01-13,2y,0.279370
...,...,...,...,...
63175,2024-01-12,2039-01-12,2y,3.424505
63176,2024-01-12,2039-01-12,3y,3.393758
63177,2024-01-12,2039-01-12,4y,3.358897
63178,2024-01-12,2039-01-12,5y,3.319531


In [12]:
ir
ir[(ir['Value Date'] == '2023-04-26') & (ir['Trade Name'] == "dummyTrade12")]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
1714644,2023-04-25,dummyTrade12,USD,-100,-508183.038348,1y,2024-04-25,10y,-51.689409
1714645,2023-04-25,dummyTrade12,USD,-50,-496480.593807,1y,2024-04-25,10y,-51.012035
1714646,2023-04-25,dummyTrade12,USD,-25,-484513.846326,1y,2024-04-25,10y,-38.148984
1714647,2023-04-25,dummyTrade12,USD,-10,-475570.883058,1y,2024-04-25,10y,-38.004951
1714648,2023-04-25,dummyTrade12,USD,-5,-472294.157998,1y,2024-04-25,10y,-39.086804
...,...,...,...,...,...,...,...,...,...
1726171,2023-04-25,dummyTrade12,USD,10,-461534.373408,15y,2038-04-27,8y,22.786971
1726172,2023-04-25,dummyTrade12,USD,25,-449348.950057,15y,2038-04-27,8y,23.471842
1726173,2023-04-25,dummyTrade12,USD,50,-425920.448809,15y,2038-04-27,8y,20.641710
1726174,2023-04-25,dummyTrade12,USD,75,-398243.736563,15y,2038-04-27,8y,21.057042


In [8]:
# dictionary storing for tenor
dummy2YCMS = {'dummyTrade1': 2, 'dummyTrade2': 2, 'dummyTrade7': 2, 'dummyTrade8': 2}
dummy5YCMS = {'dummyTrade3': 5, 'dummyTrade4': 5, 'dummyTrade9': 5, 'dummyTrade10': 5}
dummy10YCMS = {'dummyTrade5': 10, 'dummyTrade6': 10, 'dummyTrade11': 10, 'dummyTrade12': 10}
YtoInt = {2:"2y", 5: "5y", 10: "10y"}

dummyTenorCMS = {**dummy2YCMS, **dummy5YCMS, **dummy10YCMS}

In [9]:
startfilter = swap['Date'].isin(ir['Value Date']) 
timefilter = swap['Tenor'].isin(YtoInt.values())
swap_rate = swap[startfilter & timefilter]

swap_rate = swap_rate.drop_duplicates(['Date', 'Tenor'], keep = 'first') # keep the earliest quoted start date of the swap rate
swap_rate

,Date,Start Date,Tenor,Swap Rate
34263,2022-09-02,2023-09-04,10y,2.737709
34267,2022-09-02,2023-09-04,2y,2.915516
34270,2022-09-02,2023-09-04,5y,2.724626
34506,2022-09-06,2023-09-06,10y,2.853063
34510,2022-09-06,2023-09-06,2y,3.026077
...,...,...,...,...
63022,2024-01-11,2025-01-13,2y,3.051815
63025,2024-01-11,2025-01-13,5y,3.068681
63099,2024-01-12,2025-01-13,10y,3.171214
63103,2024-01-12,2025-01-13,2y,2.982736


In [10]:
swap_rate = swap_rate.drop(columns = "Start Date")
swap_rate = swap_rate.rename(columns = {"Tenor": "CMS Tenor", "Date": "Value Date"})
swap_rate

,Value Date,CMS Tenor,Swap Rate
34263,2022-09-02,10y,2.737709
34267,2022-09-02,2y,2.915516
34270,2022-09-02,5y,2.724626
34506,2022-09-06,10y,2.853063
34510,2022-09-06,2y,3.026077
...,...,...,...
63022,2024-01-11,2y,3.051815
63025,2024-01-11,5y,3.068681
63099,2024-01-12,10y,3.171214
63103,2024-01-12,2y,2.982736


In [11]:
ir['CMS Tenor'] = ir['Trade Name'].map(dummyTenorCMS).map(YtoInt)

ir = ir.merge(swap_rate, on = ['Value Date', 'CMS Tenor'], how = "left")
ir

## only map swap rate to zero shock data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,2.915516
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,2.915516
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,2y,2.915516
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,2y,2.915516
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,2y,2.915516
...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,5y,3.038587
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,5y,3.038587
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,5y,3.038587
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,5y,3.038587


In [12]:
#ir.to_csv("new ir.csv")

In [13]:
trade = trade.drop(columns = ["underlying", "pay_frequency", "maturity"])
trade.rename(columns={'trade name': 'Trade Name'}, inplace=True)
trade

,Trade Name,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,0.0044,0.0397,0.0353,4,10


In [14]:
ir = ir.merge(trade, on = "Trade Name", how = "left")

In [15]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,2.915516,0.0042,0.0379,0.0337,6,5
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,2.915516,0.0042,0.0379,0.0337,6,5
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,2y,2.915516,0.0042,0.0379,0.0337,6,5
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,2y,2.915516,0.0042,0.0379,0.0337,6,5
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,2y,2.915516,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,5y,3.038587,0.0044,0.0397,0.0353,6,10
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,5y,3.038587,0.0044,0.0397,0.0353,6,10
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,5y,3.038587,0.0044,0.0397,0.0353,6,10
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,5y,3.038587,0.0044,0.0397,0.0353,6,10


In [16]:
#ir.to_csv("complete ir.csv")
# merged both swap rate and product features data to IR

In [17]:
# feature engineering

In [18]:
ir['Swap Rate'] = ir['Swap Rate']/100

In [19]:
ir.insert(4, "Zero Rate Shock (DEC)", ir['Zero Rate Shock'] / 10000)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,0.029155,0.0042,0.0379,0.0337,6,5
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,0.029155,0.0042,0.0379,0.0337,6,5
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1y,2023-09-04,10y,4.471006,2y,0.029155,0.0042,0.0379,0.0337,6,5
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1y,2023-09-04,10y,4.333398,2y,0.029155,0.0042,0.0379,0.0337,6,5
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1y,2023-09-04,10y,5.679687,2y,0.029155,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15y,2039-01-12,8y,5.007952,5y,0.030386,0.0044,0.0397,0.0353,6,10
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15y,2039-01-12,8y,5.131505,5y,0.030386,0.0044,0.0397,0.0353,6,10
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15y,2039-01-12,8y,3.348622,5y,0.030386,0.0044,0.0397,0.0353,6,10
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15y,2039-01-12,8y,1.063634,5y,0.030386,0.0044,0.0397,0.0353,6,10


In [20]:
ir.insert(12, "Swap Rate (Shocked)", ir['Swap Rate'] + ir['Zero Rate Shock'] / 10000)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1y,2023-09-04,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1y,2023-09-04,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1y,2023-09-04,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15y,2039-01-12,8y,5.007952,5y,0.030386,0.031386,0.0044,0.0397,0.0353,6,10
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15y,2039-01-12,8y,5.131505,5y,0.030386,0.032886,0.0044,0.0397,0.0353,6,10
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15y,2039-01-12,8y,3.348622,5y,0.030386,0.035386,0.0044,0.0397,0.0353,6,10
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15y,2039-01-12,8y,1.063634,5y,0.030386,0.037886,0.0044,0.0397,0.0353,6,10


In [21]:
ir['Distance_lower (Shocked)'] = ir['Swap Rate (Shocked)'] - ir['lower_bound']
ir['Distance_upper (Shocked)'] = ir['upper_bound'] - ir['Swap Rate (Shocked)']

In [22]:
ir['Out of Bound'] = (ir['Distance_lower (Shocked)'] < 0) |  (ir['Distance_upper (Shocked)'] < 0)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1y,2023-09-04,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1y,2023-09-04,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1y,2023-09-04,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15y,2039-01-12,8y,5.007952,5y,0.030386,0.031386,0.0044,0.0397,0.0353,6,10,0.026986,0.008314,False
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15y,2039-01-12,8y,5.131505,5y,0.030386,0.032886,0.0044,0.0397,0.0353,6,10,0.028486,0.006814,False
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15y,2039-01-12,8y,3.348622,5y,0.030386,0.035386,0.0044,0.0397,0.0353,6,10,0.030986,0.004314,False
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15y,2039-01-12,8y,1.063634,5y,0.030386,0.037886,0.0044,0.0397,0.0353,6,10,0.033486,0.001814,False


In [23]:
ir['Swap_Bound Distance'] = ir[['Distance_lower (Shocked)','Distance_upper (Shocked)']].min(axis=1)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1y,2023-09-04,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False,0.014955
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1y,2023-09-04,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False,0.013745
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1y,2023-09-04,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False,0.011245
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1y,2023-09-04,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False,0.009745
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1y,2023-09-04,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False,0.009245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15y,2039-01-12,8y,5.007952,5y,0.030386,0.031386,0.0044,0.0397,0.0353,6,10,0.026986,0.008314,False,0.008314
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15y,2039-01-12,8y,5.131505,5y,0.030386,0.032886,0.0044,0.0397,0.0353,6,10,0.028486,0.006814,False,0.006814
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15y,2039-01-12,8y,3.348622,5y,0.030386,0.035386,0.0044,0.0397,0.0353,6,10,0.030986,0.004314,False,0.004314
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15y,2039-01-12,8y,1.063634,5y,0.030386,0.037886,0.0044,0.0397,0.0353,6,10,0.033486,0.001814,False,0.001814


In [24]:
df = swaption[swaption['Strike'] == 'atm']
df=df.drop('Strike',axis=1)

In [25]:
df = df.drop_duplicates(['Date', 'Expiry', 'Tenor'], keep = 'first')

In [26]:
df = df.rename(columns={"Date":"Value Date", "Expiry":"Expiry Bucket", "Tenor": "Tenor Bucket"})

In [27]:
merged = ir.merge(df, on=["Value Date", "Expiry Bucket", "Tenor Bucket"], how='left')

In [28]:
merged.loc[merged['Tenor Bucket'].str.endswith("M"), 'Tenor Bucket'] = "1.5y"
merged.loc[merged['Expiry Bucket'].str.endswith("M"), 'Expiry Bucket'] = "1.5y"
merged['Tenor Bucket'] = merged['Tenor Bucket'].str.removesuffix('y')
merged['Expiry Bucket'] = merged['Expiry Bucket'].str.removesuffix('y')
merged['Tenor Bucket'] = merged['Tenor Bucket'].astype("f8")
merged['Expiry Bucket'] = merged['Expiry Bucket'].astype("f8")
merged

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1.0,2023-09-04,10.0,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False,0.014955,0.177525
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1.0,2023-09-04,10.0,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False,0.013745,0.177525
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1.0,2023-09-04,10.0,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False,0.011245,0.177525
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1.0,2023-09-04,10.0,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False,0.009745,0.177525
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1.0,2023-09-04,10.0,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False,0.009245,0.177525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15.0,2039-01-12,8.0,5.007952,5y,0.030386,0.031386,0.0044,0.0397,0.0353,6,10,0.026986,0.008314,False,0.008314,0.253906
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15.0,2039-01-12,8.0,5.131505,5y,0.030386,0.032886,0.0044,0.0397,0.0353,6,10,0.028486,0.006814,False,0.006814,0.253906
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15.0,2039-01-12,8.0,3.348622,5y,0.030386,0.035386,0.0044,0.0397,0.0353,6,10,0.030986,0.004314,False,0.004314,0.253906
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15.0,2039-01-12,8.0,1.063634,5y,0.030386,0.037886,0.0044,0.0397,0.0353,6,10,0.033486,0.001814,False,0.001814,0.253906


In [29]:
#ir_columns = ir.columns
#mask = ~merged.columns.isin(ir_columns)
# from merged extract reordered and processed swaption csv
#new_df = merged.loc[:, mask]
#new_swaption_vol=new_df
# new_swaption.to_csv(.....)

In [32]:
merged.to_csv("../cleanedDataIR/merged.csv", index=False)